In [0]:
!pip install --upgrade tensorflow
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow import keras
import pandas as pd

In [0]:
column_names = ['reviews.rating','reviews.text']
hotel_review = pd.read_csv('http://christophe-rodrigues.fr/eval_reviews.csv', usecols=column_names, sep=";")

In [0]:
#On enleve une colonne 
#hotel_review2notes  = hotel_review.iloc[:,1:3]
#On creer un dataset avec juste les notes 1 et 5
#values = [1,5]
#hotel_reviewBis = hotel_review.loc[hotel_review['reviews.rating'].isin(values)]

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))
import pandas as pd
from sklearn.metrics import mean_squared_error
import random



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
def cleanReview(str):
  maL = tokenizer.tokenize(str)
  text_word_stops = [ _ for _ in maL if _ not in stops]
  text_word_stops = [ _.lower() for _ in text_word_stops ]
  text_lemma = [lemmatizer.lemmatize(word) for word in text_word_stops]
  text_lemma = [word for word in text_lemma  if word != "``"]
  text_lemma = [word for word in text_lemma  if word != "''"]
  #ex = nltk.pos_tag(text_lemma)
  #mesAdj = [_[0] for _ in ex if _[1]  == 'JJ'or _[1] =='JJR' or _[1] == 'JJS' or _[1] == 'NN' or _[1] == 'NNS' or _[1] == 'RB' or _[1] == 'RBS' or _[1] == 'RBR' ]
  str_lemma = ' '.join(text_lemma)
  return str_lemma


In [0]:
maListe = []
maListe2 = []
for  review in hotel_review['reviews.text']:
  maList = cleanReview(review)
  maListe2.append(maList)
for i , ele in enumerate(maListe2):
  hotel_review['reviews.text'][i] = ele

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
#Fonction qui nous retourneras le R^2
def get_score(algo, X_train,X_test,Y_train,Y_test):
    algo.fit(X_train,Y_train)
    score = algo.score(X_test,Y_test)
    return score

In [0]:
#Fonction qui nous retourneras la precision 
from sklearn.metrics import accuracy_score
def get_acc(algo,X_train,X_test,Y_train,Y_test):
    algo.fit(X_train,Y_train)
    Y_pred = algo.predict(X_test)
    acc = accuracy_score(Y_test,Y_pred) 
    return acc

In [0]:
#Fonction qui nous retourneras le mse
from sklearn.metrics import mean_squared_error 
def get_mse(algo,X_train,X_test,Y_train,Y_test):
    algo.fit(X_train,Y_train)
    Y_pred = algo.predict(X_test)
    mse = mean_squared_error(Y_test,Y_pred) 
    return mse

In [0]:
#Conversion du text en vecteurs de nombres
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)


In [0]:
import tensorflow as tf
import numpy as np

# Machine learning


In [0]:
from sklearn.model_selection import train_test_split
TrainX, TestX, TrainY, TestY = train_test_split(hotel_review['reviews.text'], hotel_review['reviews.rating'], test_size = 0.2, random_state = 0)

In [0]:
from sklearn.preprocessing import StandardScaler

#Conversion du texte en vecteurs de nombres
ConvTrainX = hub_layer(TrainX)
ConvTestX = hub_layer(TestX)

scaler = StandardScaler()
scaler.fit(ConvTrainX)#trouve la moyenne et l'ecart type
ConvTrainX = scaler.transform(ConvTrainX)#normalize
ConvTestX  =scaler.transform(ConvTestX)

#ConvTrainX

## Linear Discriminant Analysis

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

R2 = get_score(LinearDiscriminantAnalysis(),ConvTrainX,ConvTestX,TrainY,TestY)
MSE = get_mse(LinearDiscriminantAnalysis(),ConvTrainX,ConvTestX,TrainY,TestY)
print('R2 :',R2)
print('MSE :',MSE)

R2 : 0.5315
MSE : 1.209


## Random forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

R2 = get_score(RandomForestClassifier(),ConvTrainX,ConvTestX,TrainY,TestY)
MSE = get_mse(RandomForestClassifier(),ConvTrainX,ConvTestX,TrainY,TestY)
print('R2 :',R2)
print('MSE :',MSE)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


R2 : 0.512
MSE : 1.3395


## Extra tree

In [0]:
from sklearn.ensemble import ExtraTreesClassifier

R2 = get_score(ExtraTreesClassifier(),ConvTrainX,ConvTestX,TrainY,TestY)
MSE = get_mse(ExtraTreesClassifier(),ConvTrainX,ConvTestX,TrainY,TestY)
print('R2 :',R2)
print('MSE :',MSE)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


R2 : 0.495
MSE : 1.31


## Gradient Boosting Classifer

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
R2 = get_score(GradientBoostingClassifier(),ConvTrainX,ConvTestX,TrainY,TestY)
MSE = get_mse(GradientBoostingClassifier(),ConvTrainX,ConvTestX,TrainY,TestY)
print('R2 :',R2)
print('MSE :',MSE)


R2 : 0.5125
MSE : 1.295


# Deep learning

Model avec les 5 notes

In [0]:
#On retire 1 a chaque note pour le bon fonctionnement de notre futur RNN
hotel_review['reviews.rating'].loc[hotel_review['reviews.rating']==1] = 0
hotel_review['reviews.rating'].loc[hotel_review['reviews.rating']==2] = 1
hotel_review['reviews.rating'].loc[hotel_review['reviews.rating']==3] = 2
hotel_review['reviews.rating'].loc[hotel_review['reviews.rating']==4] = 3
hotel_review['reviews.rating'].loc[hotel_review['reviews.rating']==5] = 4


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
#On cree une nouvelle fois les jeux de test et train avec cette fois toutes les notes
#Separation du training set et du test set
from sklearn.model_selection import train_test_split
TrainX2, TestX2, TrainY2, TestY2 = train_test_split(hotel_review['reviews.text'], hotel_review['reviews.rating'], test_size = 0.4, random_state = 0)

In [0]:
TrainX.shape, TestX.shape, TrainY.shape, TestY.shape 

((8000,), (2000,), (8000,), (2000,))

In [0]:
TrainX2.shape, TestX2.shape, TrainY2.shape, TestY2.shape 

((6000,), (4000,), (6000,), (4000,))

In [0]:
#Construction du reseau de neurones avec 5 output
model2 = tf.keras.Sequential()
model2.add(hub_layer)
model2.add(tf.keras.layers.Dense(10, activation='relu'))
model2.add(tf.keras.layers.Dense(10, activation='relu'))
model2.add(tf.keras.layers.Dense(5, activation='softmax'))

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 55        
Total params: 400,395
Trainable params: 400,395
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Choix de l'algo de correction de l'erreur
#Ici on choisi sparse_categorical_crossentropy comme loss puisque notre output sera constitué de plusieurs categories.
model2.compile(optimizer='adam',loss ='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
#On convertit les jeux de donnees en format tf.tensor pour que le format des donnees soit le plus adapte à notre RNN.
TrainX2 = tf.convert_to_tensor(TrainX2, dtype=tf.string)
TrainY2 = tf.convert_to_tensor(TrainY2, dtype=tf.int32)
TestX2 = tf.convert_to_tensor(TestX2, dtype=tf.string)
TestY2 = tf.convert_to_tensor(TestY2, dtype=tf.int32)

In [0]:
#Apprentissage du modele sur 15 époques
history = model2.fit(TrainX2,TrainY2,epochs=15,verbose=1,steps_per_epoch=len(TrainX2))

Train on 6000 samples
Epoch 1/15
6000/6000 [==============================] - 32s 5ms/sample - loss: 1.0851 - accuracy: 0.5138
Epoch 2/15
6000/6000 [==============================] - 31s 5ms/sample - loss: 0.8741 - accuracy: 0.6130
Epoch 3/15
6000/6000 [==============================] - 31s 5ms/sample - loss: 0.7591 - accuracy: 0.6690
Epoch 4/15
6000/6000 [==============================] - 31s 5ms/sample - loss: 0.6638 - accuracy: 0.7162
Epoch 5/15
6000/6000 [==============================] - 31s 5ms/sample - loss: 0.5755 - accuracy: 0.7640
Epoch 6/15
6000/6000 [==============================] - 31s 5ms/sample - loss: 0.4888 - accuracy: 0.8112
Epoch 7/15
6000/6000 [==============================] - 31s 5ms/sample - loss: 0.4085 - accuracy: 0.8480
Epoch 8/15
6000/6000 [==============================] - 32s 5ms/sample - loss: 0.3368 - accuracy: 0.8733
Epoch 9/15
6000/6000 [==============================] - 32s 5ms/sample - loss: 0.2747 - accuracy: 0.9035
Epoch 10/15
6000/6000 [==========

In [0]:
#Evaluation du modele sur le jeu de test
test_loss2, test_acc2 = model2.evaluate(TestX2,  TestY2, verbose=2)

print('\nTest accuracy:', test_acc2)

4000/1 - 0s - loss: 2.8741 - accuracy: 0.5565

Test accuracy: 0.5565


In [0]:
import numpy as np

prediction = model2.predict(TestX2)
new_prediction = np.zeros(len(prediction))
for i in range(len(prediction)):
   new_prediction[i] = np.where(prediction[i]==max(prediction[i]))[0][0]+1
mean_squared_error(TestY2,new_prediction)

1.9685

In [0]:
#Pas forcement utile pour le projet, c'etait pour essayer plusieurs modeles 

# def create_model(neural_hidden_layer):
#   model2 = tf.keras.Sequential()
#   model2.add(hub_layer)
#   model2.add(tf.keras.layers.Dense(neural_hidden_layer, activation='relu'))
#   model2.add(tf.keras.layers.Dense(5, activation='softmax'))
#   return model2



# import pandas as pd
# result = []
# for neural in range(10,15):
#   monModel = create_model(neural)
#   monModel.compile(optimizer='adam',loss ='sparse_categorical_crossentropy',metrics=['accuracy'])
#   for epoch in range(10,15):
#     monModel.fit(TrainX2,TrainY2,epochs=epoch,verbose=2)
#     test_loss2, test_acc2 = model2.evaluate(TestX2,  TestY2, verbose=2)
#     result.append({'neural':neural,'epoch':epoch,'accuracy':test_acc2,'loss':test_loss2})
# result = pd.DataFrame(result)

# result

# result.loc[result['accuracy']==result.accuracy.max()]

## TF IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [0]:
#another technique to transfort the texts in vector using tf-idf
X_tfidf = vectorizer.fit_transform(X)
Y_tfidf = Y
X_tfidf.shape

In [0]:
xTrain4, xTest4, yTrain4, yTest4 = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [0]:
#another technique to transfort the texts in vector using tf-idf
xTrain4 = vectorizer.fit_transform(xTrain4).todense()
xTest4 = vectorizer.transform(xTest4).todense()


In [0]:
xTrain4.shape , yTrain4.shape , xTest4.shape , yTest4.shape

In [0]:
from keras.layers.core import Dense, Dropout, Activation

In [0]:
#Construction du reseau de neurones avec 5 output
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Dense(12, input_dim= 19003, activation='relu'))
model2.add(tf.keras.layers.Dense(12, activation='relu'))
model2.add(tf.keras.layers.Dense(1, activation='softmax'))
model2.summary()

In [0]:
model2.compile(optimizer='adam',loss ='mse',metrics=['mse','accuracy'])

In [0]:
xTrain4 = tf.convert_to_tensor(xTrain4, dtype=tf.float32)
#yTrain4 = tf.convert_to_tensor(yTrain4, dtype=tf.int32)
xTest4 = tf.convert_to_tensor(xTest4, dtype=tf.float32)
#yTest4 = tf.convert_to_tensor(yTest4, dtype=tf.int32)

In [0]:
history = model2.fit(xTrain4,yTrain4,epochs=15,verbose=1,steps_per_epoch=int(xTrain4.shape[0]))

In [0]:
#Evaluation du modele sur le jeu de test
test_loss4, test_mse4, test_acc4 = model2.evaluate(xTest4,  yTest4,steps=int(xTest4.shape[0]))

print('\nTest accuracy:', test_acc4)
print('\nTest accuracy:', test_mse4)

In [0]:
pip install --upgrade tensorflow==1.14